In [44]:
!pip install sentencepiece
!pip install transformers
!pip install rich[jupyter]

In [45]:
import pandas as pd
df = pd.read_csv("/content/train.csv")

In [46]:
df.sample(3)

,answer,paragraph_question,question,sentence,paragraph,sentence_answer,paragraph_answer,paragraph_sentence
521,Phase Breaks,question: What invention prevented lines from ...,What invention prevented lines from getting mi...,This is achieved by Neutral Sections (also kno...,To prevent the risk of out-of-phase supplies m...,This is achieved by Neutral Sections (also kno...,To prevent the risk of out-of-phase supplies m...,To prevent the risk of out-of-phase supplies m...
737,"Naples, Sicily, and Savoy, although sided with...",question: Which side of the coalitions did Nap...,"Which side of the coalitions did Naples, Sicil...","Naples, Sicily, and Savoy, although sided with...","Many middle and small powers in Europe, unlike...","<hl> Naples, Sicily, and Savoy, although sided...","Many middle and small powers in Europe, unlike...","Many middle and small powers in Europe, unlike..."
740,2001,question: When did a Close Encounters DVD come...,When did a Close Encounters DVD come out?,"Nevertheless, the re-release was a moderate su...",Spielberg then revisited his Close Encounters ...,"Nevertheless, the re-release was a moderate su...",Spielberg then revisited his Close Encounters ...,Spielberg then revisited his Close Encounters ...


In [47]:
df["paragraph"] = "Generate Question-Answer pairs for the given paragraph: "+df["paragraph"]

In [48]:
df['QA'] = 'Question: '+df['question']+', Answer: '+df['answer']

In [49]:
df.head()

,answer,paragraph_question,question,sentence,paragraph,sentence_answer,paragraph_answer,paragraph_sentence,QA
0,high,question: How does Boston rank on world livabi...,How does Boston rank on world livability?,The area's many colleges and universities make...,Generate Question-Answer pairs for the given p...,The area's many colleges and universities make...,The area's many colleges and universities make...,<hl> The area's many colleges and universities...,Question: How does Boston rank on world livabi...
1,mandate to rule that region as a national colony.,question: What do European powers that exibit ...,What do European powers that exibit their cont...,The European power that demonstrated its contr...,Generate Question-Answer pairs for the given p...,The European power that demonstrated its contr...,The major European powers laid claim to the ar...,The major European powers laid claim to the ar...,Question: What do European powers that exibit ...
2,Khorasan,"question: Where was Nader Shan from?, context:...",Where was Nader Shan from?,"In 1729, Nader Shah, a chieftain and military ...",Generate Question-Answer pairs for the given p...,"In 1729, Nader Shah, a chieftain and military ...","In 1729, Nader Shah, a chieftain and military ...","<hl> In 1729, Nader Shah, a chieftain and mili...","Question: Where was Nader Shan from?, Answer: ..."
3,multi-ethnic and multi-lingual,question: the holy roman empire due to its man...,the holy roman empire due to its many differen...,It had a multi-ethnic and multi-lingual struct...,Generate Question-Answer pairs for the given p...,It had a <hl> multi-ethnic and multi-lingual <...,"At the same time, naval innovations led to a G...","At the same time, naval innovations led to a G...",Question: the holy roman empire due to its man...
4,rib and plank,question: What new shipbuilding method was inv...,What new shipbuilding method was invented in t...,Shipbuilding improved with the use of the rib ...,Generate Question-Answer pairs for the given p...,Shipbuilding improved with the use of the <hl>...,The construction of cathedrals and castles adv...,The construction of cathedrals and castles adv...,Question: What new shipbuilding method was inv...


In [50]:
df = df[['QA','paragraph']]

In [51]:
df

,QA,paragraph
0,Question: How does Boston rank on world livabi...,Generate Question-Answer pairs for the given p...
1,Question: What do European powers that exibit ...,Generate Question-Answer pairs for the given p...
2,"Question: Where was Nader Shan from?, Answer: ...",Generate Question-Answer pairs for the given p...
3,Question: the holy roman empire due to its man...,Generate Question-Answer pairs for the given p...
4,Question: What new shipbuilding method was inv...,Generate Question-Answer pairs for the given p...
...,...,...
995,Question: What is a unit of work called in a d...,Generate Question-Answer pairs for the given p...
996,Question: What part of the US military develop...,Generate Question-Answer pairs for the given p...
997,Question: What did Sony change about the PS3 g...,Generate Question-Answer pairs for the given p...
998,Question: How can users utilize the Kinect to ...,Generate Question-Answer pairs for the given p...


In [52]:
# Importing libraries
import os
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import os

# Importing the T5 modules from huggingface/transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration

from rich.table import Column, Table
from rich import box
from rich.console import Console

# define a rich console logger
console=Console(record=True)

def display_df(df):
  """display dataframe in ASCII format"""

  console=Console()
  table = Table(Column("source_text", justify="center" ), Column("target_text", justify="center"), title="Sample Data",pad_edge=False, box=box.ASCII)

  for i, row in enumerate(df.values.tolist()):
    table.add_row(row[0], row[1])

  console.print(table)

training_logger = Table(Column("Epoch", justify="center" ),
                        Column("Steps", justify="center"),
                        Column("Loss", justify="center"),
                        title="Training Status",pad_edge=False, box=box.ASCII)


In [53]:
# Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [54]:
class YourDataSetClass(Dataset):
  """
  Creating a custom dataset for reading the dataset and
  loading it into the dataloader to pass it to the neural network for finetuning the model

  """

  def __init__(self, dataframe, tokenizer, source_len, target_len, source_text, target_text):
    self.tokenizer = tokenizer
    self.data = dataframe
    self.source_len = source_len
    self.summ_len = target_len
    self.target_text = self.data[target_text]
    self.source_text = self.data[source_text]

  def __len__(self):
    return len(self.target_text)

  def __getitem__(self, index):
    source_text = str(self.source_text[index])
    target_text = str(self.target_text[index])

    #cleaning data so as to ensure data is in string type
    source_text = ' '.join(source_text.split())
    target_text = ' '.join(target_text.split())

    source = self.tokenizer.batch_encode_plus([source_text], max_length= self.source_len, pad_to_max_length=True, truncation=True, padding="max_length", return_tensors='pt')
    target = self.tokenizer.batch_encode_plus([target_text], max_length= self.summ_len, pad_to_max_length=True, truncation=True, padding="max_length", return_tensors='pt')

    source_ids = source['input_ids'].squeeze()
    source_mask = source['attention_mask'].squeeze()
    target_ids = target['input_ids'].squeeze()
    target_mask = target['attention_mask'].squeeze()

    return {
        'source_ids': source_ids.to(dtype=torch.long),
        'source_mask': source_mask.to(dtype=torch.long),
        'target_ids': target_ids.to(dtype=torch.long),
        'target_ids_y': target_ids.to(dtype=torch.long)
    }

In [55]:
def train(epoch, tokenizer, model, device, loader, optimizer):

  """
  Function to be called for training with the parameters passed from main function

  """

  model.train()
  for _,data in enumerate(loader, 0):
    y = data['target_ids'].to(device, dtype = torch.long)
    y_ids = y[:, :-1].contiguous()
    lm_labels = y[:, 1:].clone().detach()
    lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
    ids = data['source_ids'].to(device, dtype = torch.long)
    mask = data['source_mask'].to(device, dtype = torch.long)

    outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, labels=lm_labels)
    loss = outputs[0]

    if _%10==0:
      training_logger.add_row(str(epoch), str(_), str(loss))
      console.print(training_logger)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [56]:
def validate(epoch, tokenizer, model, device, loader):

  """
  Function to evaluate model for predictions

  """
  model.eval()
  predictions = []
  actuals = []
  with torch.no_grad():
      for _, data in enumerate(loader, 0):
          y = data['target_ids'].to(device, dtype = torch.long)
          ids = data['source_ids'].to(device, dtype = torch.long)
          mask = data['source_mask'].to(device, dtype = torch.long)

          generated_ids = model.generate(
              input_ids = ids,
              attention_mask = mask,
              max_length=150,
              num_beams=2,
              repetition_penalty=2.5,
              length_penalty=1.0,
              early_stopping=True
              )
          preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
          target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
          if _%10==0:
              console.print(f'Completed {_}')

          predictions.extend(preds)
          actuals.extend(target)
  return predictions, actuals

In [57]:
def T5Trainer(dataframe, source_text, target_text, model_params, output_dir="./outputs/" ):

  """
  T5 trainer

  """

  # Set random seeds and deterministic pytorch for reproducibility
  torch.manual_seed(model_params["SEED"]) # pytorch random seed
  np.random.seed(model_params["SEED"]) # numpy random seed
  torch.backends.cudnn.deterministic = True

  # logging
  console.log(f"""[Model]: Loading {model_params["MODEL"]}...\n""")

  # tokenzier for encoding the text
  tokenizer = T5Tokenizer.from_pretrained(model_params["MODEL"])

  # Defining the model. We are using t5-base model and added a Language model layer on top for generation of Summary.
  # Further this model is sent to device (GPU/TPU) for using the hardware.
  model = T5ForConditionalGeneration.from_pretrained(model_params["MODEL"])
  model = model.to(device)

  # logging
  console.log(f"[Data]: Reading data...\n")

  # Importing the raw dataset
  dataframe = dataframe[[source_text,target_text]]
  display_df(dataframe.head(2))


  # Creation of Dataset and Dataloader
  # Defining the train size. So 80% of the data will be used for training and the rest for validation.
  train_size = 0.8
  train_dataset=dataframe.sample(frac=train_size,random_state = model_params["SEED"])
  val_dataset=dataframe.drop(train_dataset.index).reset_index(drop=True)
  train_dataset = train_dataset.reset_index(drop=True)

  console.print(f"FULL Dataset: {dataframe.shape}")
  console.print(f"TRAIN Dataset: {train_dataset.shape}")
  console.print(f"TEST Dataset: {val_dataset.shape}\n")


  # Creating the Training and Validation dataset for further creation of Dataloader
  training_set = YourDataSetClass(train_dataset, tokenizer, model_params["MAX_SOURCE_TEXT_LENGTH"], model_params["MAX_TARGET_TEXT_LENGTH"], source_text, target_text)
  val_set = YourDataSetClass(val_dataset, tokenizer, model_params["MAX_SOURCE_TEXT_LENGTH"], model_params["MAX_TARGET_TEXT_LENGTH"], source_text, target_text)


  # Defining the parameters for creation of dataloaders
  train_params = {
      'batch_size': model_params["TRAIN_BATCH_SIZE"],
      'shuffle': True,
      'num_workers': 0
      }


  val_params = {
      'batch_size': model_params["VALID_BATCH_SIZE"],
      'shuffle': False,
      'num_workers': 0
      }


  # Creation of Dataloaders for testing and validation. This will be used down for training and validation stage for the model.
  training_loader = DataLoader(training_set, **train_params)
  val_loader = DataLoader(val_set, **val_params)


  # Defining the optimizer that will be used to tune the weights of the network in the training session.
  optimizer = torch.optim.Adam(params =  model.parameters(), lr=model_params["LEARNING_RATE"])


  # Training loop
  console.log(f'[Initiating Fine Tuning]...\n')

  for epoch in range(model_params["TRAIN_EPOCHS"]):
      train(epoch, tokenizer, model, device, training_loader, optimizer)

  console.log(f"[Saving Model]...\n")
  #Saving the model after training
  path = os.path.join(output_dir, "model_files")
  model.save_pretrained(path)
  tokenizer.save_pretrained(path)


  # evaluating test dataset
  console.log(f"[Initiating Validation]...\n")
  for epoch in range(model_params["VAL_EPOCHS"]):
    predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
    final_df = pd.DataFrame({'Generated Text':predictions,'Actual Text':actuals})
    final_df.to_csv(os.path.join(output_dir,'predictions.csv'))

  console.save_text(os.path.join(output_dir,'logs.txt'))

  console.log(f"[Validation Completed.]\n")
  console.print(f"""[Model] Model saved @ {os.path.join(output_dir, "model_files")}\n""")
  console.print(f"""[Validation] Generation on Validation data saved @ {os.path.join(output_dir,'predictions.csv')}\n""")
  console.print(f"""[Logs] Logs saved @ {os.path.join(output_dir,'logs.txt')}\n""")

In [64]:
model_params={
    "MODEL":"t5-base",             # model_type: t5-base/t5-large
    "TRAIN_BATCH_SIZE":8,          # training b
    "TRAIN_EPOCHS":1,              # number of training epochs
    "VAL_EPOCHS":1,                # number of validation epochs
    "LEARNING_RATE":1e-4,          # learatch size
    "VALID_BATCH_SIZE":8,          # validation batch sizening rate
    "MAX_SOURCE_TEXT_LENGTH":512,  # max length of source text
    "MAX_TARGET_TEXT_LENGTH":50,   # max length of target text
    "SEED": 42                     # set seed for reproducibility

}

In [65]:
T5Trainer(dataframe=df[:500], source_text="paragraph", target_text="QA", model_params=model_params, output_dir="outputs")

[18:48:55] [Model]: Loading t5-base...                                           <ipython-input-57-b1eef7f6be59>:14
                                                                                                                   

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


[18:48:57] [Data]: Reading data...                                               <ipython-input-57-b1eef7f6be59>:25
                                                                                                                   

                                                    Sample Data                                                    
+-----------------------------------------------------------------------------------------------------------------+
|                      source_text                       |                       target_text                      |
|--------------------------------------------------------+--------------------------------------------------------|
|Generate Question-Answer pairs for the given paragraph: |  Question: How does Boston rank on world livability?,  |
| The area's many colleges and universities make Boston  |                      Answer: high                      |
|    an international center of higher education and     |                                                        |
|  medicine, and the city is considered to be a world    |                                                        |
|   leader in innovation. Boston's economic base also    |                                                        |
| includes finance, professional and business services,  |                                                        |
| biotechnology, information technology, and government  |                                                        |
| activities. Households in the city claim the highest   |                                                        |
|  average rate of philanthropy in the United States;    |                                                        |
|businesses and institutions rank amongst the top in the |                                                        |
|     country for environmental sustainability and       |                                                        |
| investment. The city has one of the highest costs of   |                                                        |
|living in the United States, though it remains high on  |                                                        |
|              world livability rankings.                |                                                        |
|Generate Question-Answer pairs for the given paragraph: |   Question: What do European powers that exibit their  |
| The major European powers laid claim to the areas of   |  control ovet an area accept?, Answer: mandate to rule |
| Africa where they could exhibit a sphere of influence  |            that region as a national colony.           |
| over the area. These claims did not have to have any   |                                                        |
|substantial land holdings or treaties to be legitimate. |                                                        |
|The European power that demonstrated its control over a |                                                        |
|territory accepted the mandate to rule that region as a |                                                        |
|  national colony. The European nation that held the    |                                                        |
|   claim developed and benefited from their colony’s    |                                                        |
|   commercial interests without having to fear rival    |                                                        |
|European competition. With the colonial claim came the  |                                                        |
|  underlying assumption that the European power that    |                                                        |
|    exerted control would use its mandate to offer      |                                                        |
|    protection and provide welfare for its colonial     |                                                        |
| peoples, however, this principle remained more theory  |                                                        |
|than practice. There were many documented instances of  |                                                        |
|material and moral conditions deteriorating for native 

FULL Dataset: (500, 2)

...

Completed 10

[18:50:07] [Validation Completed.]                                               <ipython-input-57-b1eef7f6be59>:95
                                                                                                                   

[Model] Model saved @ outputs/model_files

[Validation] Generation on Validation data saved @ outputs/predictions.csv

[Logs] Logs saved @ outputs/logs.txt